In [1]:
pip install pyomo pandas openpyxl


  Obtaining dependency information for pyomo from https://files.pythonhosted.org/packages/d1/dc/139e49cee5b003e32bd38d222dfd8ced549078c98809f88f736abd3d7650/Pyomo-6.8.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/a8/44/d9502bf0ed197ba9bf1103c9867d5904ddcaf869e52329787fc54ed70cc8/pandas-2.2.3-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Using cached pandas-2.2.3-cp311-cp311-macosx_10_9_x86_64.whl.metadata (89 kB)
  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for ply from https://files.pythonhosted.org/packages/a3/58/35da89ee790598a0700ea49b2a66594140f44dec458c07e8e3d4979137fc/ply-3.11-py2.py3-none-any.whl.metadata
  Obtaining dependency information for numpy>=1.23.2 from https://files.pythonhosted.org/pack

In [2]:
import pandas as pd
from pyomo.environ import *
from datetime import datetime, timedelta

In [352]:
# Load data from Excel
teams_df = pd.read_excel('schedule_data.xlsx', sheet_name='Teams')
# Load data from Excel with date parsing
grounds_df = pd.read_excel(
    'schedule_data.xlsx',
    sheet_name='Grounds',
    parse_dates=['Date'],
)
dates_sorted = sorted(grounds_df['Date'].tolist())

teams = teams_df['TeamName'].tolist()

In [153]:
grounds_df

,Date,Kerava,Tikkurila,Käpylä
0,2024-08-03,1,0,0
1,2024-08-02,1,0,0
2,2024-07-30,1,0,0
3,2024-07-29,1,0,0
4,2024-07-28,1,0,0
5,2024-07-27,1,0,0
6,2024-07-25,1,0,0
7,2024-07-24,1,0,0
8,2024-07-23,1,0,0
9,2024-07-22,1,0,0


In [154]:
teams

['Empire Lions',
 'Greater Helsinki Markhors',
 'The J Team Jaguars',
 'Helsinki Titans',
 'TreCC Amperes',
 'Vantaa Legends',
 'SKK Rapids',
 'BTCC Thundercats']

In [155]:
# Extract dates and grounds
dates = pd.to_datetime(grounds_df['Date'], format='%m/%d/%Y').tolist()
grounds = list(grounds_df.columns)
grounds.remove('Date')  # Remove the 'Date' column to get ground names

In [156]:
grounds

['Kerava', 'Tikkurila', 'Käpylä']

In [444]:
# Initialize a dictionary to hold ground capacities
ground_capacity = {}

for idx, row in grounds_df.iterrows():
    date = row['Date']
    is_weekend_flag = row['IsWeekend']
    for ground in grounds:
        if row[ground] == 1:
            if is_weekend_flag:
                capacity = 3  # 3 matches on weekends
            else:
                capacity = 1  # 1 match on weekdays
            ground_capacity[(ground, date)] = capacity
        else:
            ground_capacity[(ground, date)] = 0  # Ground unavailable



In [445]:
ground_capacity

{('Kerava', Timestamp('2024-08-03 00:00:00')): 3,
 ('Tikkurila', Timestamp('2024-08-03 00:00:00')): 0,
 ('Käpylä', Timestamp('2024-08-03 00:00:00')): 0,
 ('Kerava', Timestamp('2024-08-02 00:00:00')): 1,
 ('Tikkurila', Timestamp('2024-08-02 00:00:00')): 0,
 ('Käpylä', Timestamp('2024-08-02 00:00:00')): 0,
 ('Kerava', Timestamp('2024-07-30 00:00:00')): 1,
 ('Tikkurila', Timestamp('2024-07-30 00:00:00')): 0,
 ('Käpylä', Timestamp('2024-07-30 00:00:00')): 0,
 ('Kerava', Timestamp('2024-07-29 00:00:00')): 1,
 ('Tikkurila', Timestamp('2024-07-29 00:00:00')): 0,
 ('Käpylä', Timestamp('2024-07-29 00:00:00')): 0,
 ('Kerava', Timestamp('2024-07-28 00:00:00')): 3,
 ('Tikkurila', Timestamp('2024-07-28 00:00:00')): 0,
 ('Käpylä', Timestamp('2024-07-28 00:00:00')): 0,
 ('Kerava', Timestamp('2024-07-27 00:00:00')): 3,
 ('Tikkurila', Timestamp('2024-07-27 00:00:00')): 0,
 ('Käpylä', Timestamp('2024-07-27 00:00:00')): 0,
 ('Kerava', Timestamp('2024-07-25 00:00:00')): 1,
 ('Tikkurila', Timestamp('2024-0

In [469]:
# Function to determine if a date is a weekend
def is_weekend(date):
    return 1 if date.weekday() >= 5 else 0  # 1 for Saturday/Sunday, 0 otherwise

# Add a column to indicate weekend
grounds_df['IsWeekend'] = grounds_df['Date'].apply(is_weekend)


In [470]:
from itertools import permutations

# Generate all ordered pairs (home, away) excluding self-matches
matches = [(h, a) for h in teams for a in teams if h != a]


In [471]:
# Initialize the model
model = ConcreteModel()

# Sets
model.Teams = Set(initialize=teams)
model.Dates = Set(initialize=dates_sorted)
model.Grounds = Set(initialize=grounds)
model.Matches = Set(initialize=matches, dimen=2)

# Ground Availability Parameter
model.GroundAvailability = Param(model.Grounds, model.Dates, initialize=ground_capacity, default=0)

# Determine if a date is weekend
date_weekend = {date: is_weekend(date) for date in dates_sorted}
model.IsWeekend = Param(model.Dates, initialize=date_weekend, within=Binary)

# Map each date to its ISO week number
date_to_week = {date: date.isocalendar()[1] for date in dates_sorted}

# Create a set of unique weeks
weeks = sorted(set(date_to_week.values()))
model.Weeks = Set(initialize=weeks)

# Parameter to map dates to weeks
model.DateToWeek = Param(model.Dates, initialize=date_to_week, within=NonNegativeIntegers)


# Binary variable: y[h, a, d, g] = 1 if match (h, a) is scheduled on date d at ground g
model.y = Var(model.Matches, model.Dates, model.Grounds, domain=Binary)



In [472]:
special_team = 'TreCC Amperes'
regular_teams = [t for t in teams if t != special_team]

In [473]:
def match_scheduled_once_rule(model, h, a):
    return sum(model.y[h, a, d, g] for d in model.Dates for g in model.Grounds) == 1
model.MatchScheduledOnce = Constraint(model.Matches, rule=match_scheduled_once_rule)

def ground_capacity_rule(model, g, d):
    return sum(model.y[h, a, d, g] for h, a in model.Matches) <= model.GroundAvailability[g, d]
model.GroundCapacityConstraint = Constraint(model.Grounds, model.Dates, rule=ground_capacity_rule)

def team_two_match_per_day_rule(model, t, d):
    return sum(model.y[h, a, d, g] for h, a in model.Matches if h == t or a == t for g in model.Grounds) <= 2
model.TeamTwoMatchPerDay = Constraint(model.Teams, model.Dates, rule=team_two_match_per_day_rule)



In [474]:
# 4. No Team Plays More Than Two Matches Per Week
def team_max_two_matches_per_week_rule(model, t, w):
    # Sum of matches for team t in week w
    return sum(model.y[h, a, d, g] 
               for d in model.Dates if model.DateToWeek[d] == w 
               for g in model.Grounds 
               for (h, a) in model.Matches if h == t or a == t) <= 2
model.TeamMaxTwoMatchesPerWeek = Constraint(model.Teams, model.Weeks, rule=team_max_two_matches_per_week_rule)


In [475]:
# 5. TreCC Amperes: Matches Only on Weekends
def trecc_weekend_only_rule(model, h, a, d, g):
    if h == special_team or a == special_team:
        return model.y[h, a, d, g] <= model.IsWeekend[d]
    else:
        return Constraint.Skip
model.TreccWeekendOnly = Constraint(model.Matches, model.Dates, model.Grounds, rule=trecc_weekend_only_rule)

# 6. TreCC Amperes: No More Than 2 Matches Per Weekend Day
def trecc_max_two_matches_per_day_rule(model, d):
    if model.IsWeekend[d] == 1:
        return sum(model.y[h, a, d, g] 
                   for h, a in model.Matches if h == special_team or a == special_team 
                   for g in model.Grounds) <= 2
    else:
        return Constraint.Skip
model.TreccMaxTwoMatchesPerDay = Constraint(model.Dates, rule=trecc_max_two_matches_per_day_rule)



In [476]:
model.Objective = Objective(expr=0, sense=minimize)


In [477]:
# Choose the solver
solver = SolverFactory('glpk',executable='/usr/local/bin/glpsol')  

# Solve the model
result = solver.solve(model, tee=True)


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmp1no49076.glpk.raw
 --wglp /var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmp1oocxvzw.glpk.glp
 --cpxlp /var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmp711l316x.pyomo.lp
Reading problem data from '/var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmp711l316x.pyomo.lp'...
/var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmp711l316x.pyomo.lp:57507: warning: lower bound of variable 'x4' redefined
/var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmp711l316x.pyomo.lp:57507: warning: upper bound of variable 'x4' redefined
2323 rows, 6889 columns, 43638 non-zeros
6888 integer variables, all of which are binary
64395 lines were read
Writing problem data to '/var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmp1oocxvzw.glpk.glp'...
55177 lines were written
GLPK Integer Optimizer 5.0
2323 rows, 6889 columns, 43638 non-zeros
6888 integer variables, all of whic

In [478]:
from pyomo.opt import SolverStatus, TerminationCondition

# Check if the solution is feasible
if (result.solver.status == SolverStatus.ok) and (result.solver.termination_condition == TerminationCondition.optimal):
    # Retrieve the schedule
    schedule = []
    for h, a in model.Matches:
        for d in model.Dates:
            for g in model.Grounds:
                if value(model.y[h, a, d, g]) == 1:
                    schedule.append({
                        'HomeTeam': h,
                        'AwayTeam': a,
                        'Date': d.strftime('%d-%m-%Y'),
                        'Ground': g
                    })
    schedule_df = pd.DataFrame(schedule)
    schedule_df['Date'] = pd.to_datetime(schedule_df['Date'], format='%d-%m-%Y')

    # Sort the schedule by date
    schedule_df.sort_values(by='Date', inplace=True)
    
    # Reset index
    schedule_df.reset_index(drop=True, inplace=True)
    
    # Display the schedule
    print(schedule_df)
    
    # Optionally, export to Excel
    #schedule_df.to_excel('schedule_output.xlsx', index=False)
else:
    print('No feasible solution found.')
    print('Solver Status:', result.solver.status)
    print('Termination Condition:', result.solver.termination_condition)


                     HomeTeam                   AwayTeam       Date  Ground
0              Vantaa Legends               Empire Lions 2024-06-01  Kerava
1                Empire Lions              TreCC Amperes 2024-06-01  Kerava
2            BTCC Thundercats            Helsinki Titans 2024-06-01  Kerava
3             Helsinki Titans              TreCC Amperes 2024-06-02  Kerava
4                  SKK Rapids         The J Team Jaguars 2024-06-02  Kerava
5          The J Team Jaguars             Vantaa Legends 2024-06-02  Kerava
6                Empire Lions             Vantaa Legends 2024-06-04  Kerava
7   Greater Helsinki Markhors         The J Team Jaguars 2024-06-05  Kerava
8                  SKK Rapids            Helsinki Titans 2024-06-07  Kerava
9   Greater Helsinki Markhors           BTCC Thundercats 2024-06-08  Kerava
10               Empire Lions            Helsinki Titans 2024-06-08  Kerava
11             Vantaa Legends                 SKK Rapids 2024-06-09  Kerava
12          